In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

In [ ]:
load_dotenv()

In [ ]:
pg_user = os.getenv("POSTGRES_USER", "root")
pg_password = os.getenv("POSTGRES_PASSWORD", "<PASSWORD>")
pg_host = os.getenv("POSTGRES_HOST", "localhost")
pg_port = os.getenv("POSTGRES_PORT", "5432")
pg_db = os.getenv("POSTGRES_DB", "postgres")
month = os.getenv("MONTH", 1)
year = os.getenv("YEAR", 2021)
target_table = os.getenv("TARGET_TABLE", "yellow_taxi_data")
chunksize = os.getenv("CHUNKSIZE", 100)

In [ ]:
dtype = {
    "VendorID": "Int64",
    "passenger_count": "Int64",
    "trip_distance": "float64",
    "RatecodeID": "Int64",
    "store_and_fwd_flag": "string",
    "PULocationID": "Int64",
    "DOLocationID": "Int64",
    "payment_type": "Int64",
    "fare_amount": "float64",
    "extra": "float64",
    "mta_tax": "float64",
    "tip_amount": "float64",
    "tolls_amount": "float64",
    "improvement_surcharge": "float64",
    "total_amount": "float64",
    "congestion_surcharge": "float64"
}

In [ ]:
parse_dates = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
]

In [ ]:
prefix = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/"

In [ ]:
df_iter = pd.read_csv(
    f'{prefix}/yellow_tripdata_{year}-{int(month):02d}.csv.gz',
    dtype=dtype,
    parse_dates=parse_dates,
    iterator=True,
    chunksize=int(chunksize)
)
len(next(df_iter))

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine(f'postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}')

In [ ]:
print(pd.io.sql.get_schema(next(df_iter), name='yellow_taxi_data', con=engine))

In [ ]:
first = True
for df in tqdm(df_iter):
    if first:
        df.head(0).to_sql(target_table, engine, if_exists='replace')
        first = False
        print(f"Created table {target_table}!")

    df.to_sql(target_table, engine, if_exists="append")
    print("Inserted chunk: ", len(df))